In [ ]:
from scipy.sparse import csc_array
from scipy.sparse.linalg import bicgstab
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import gmres
import numpy as np
import matplotlib.pyplot as plt
pi = np.pi
from copy import deepcopy

In [ ]:
def DatosMat(path):
    data = [[],[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

def DatosVec(path):
    data = [[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

def Kaczmarz(A,N,phi,x0,max_iter,tol):
    x = x0
    r = phi - A.dot(x)
    err = 1
    k = 0
    while k<max_iter and err>tol:
        for i in range(N):
            v = np.eye(1,N,i).reshape(N)
            Av = A @ v
            alpha = np.vdot(Av,r) / np.vdot(Av,Av)
            x += alpha * v
            r = phi - A.dot(x)
        err = np.sqrt(np.real(np.vdot(r,r)))
        if err<tol:
            print("Kaczmarz for D converged in {0} iterations. Error {1}".format(k+1,err))
            return x
        k += 1
    print("Kaczmarz for D did NOT converge in {0} iterations. Error {1}".format(max_iter,err))
    return x      

In [ ]:
Nx, Nt = 20, 20
block_x, block_t = 4, 4
x_elements, t_elements = int(Nx/block_x), int(Nt/block_t)
Ntest, Nagg = 10, 2*(block_x*block_t)
Ntot = Nx*Nt
Coords = np.zeros((Nx,Nt,2),dtype=int)
for x in range(Nx):
    for t in range(Nt):
        for s in range(2):
            Coords[x,t,s] = x * Nt * 2 + t * 2 + s
XCoord = np.zeros(2*Nx*Nt,dtype=int)
TCoord = np.zeros(2*Nx*Nt,dtype=int)
SCoord = np.zeros(2*Nx*Nt,dtype=int)
Agg = np.zeros((2*block_x*block_t,x_elements*t_elements),dtype=int)
print("Lattice dimensions Nx={0}, Nt={1}".format(Nx,Nt))
print("Number of aggregates {0}".format(Nagg))
print("Block_x={0}, Block_t={1}".format(block_x,block_t))
print("Number of test vectors {0}".format(Ntest))
print("Matrix dimension {0}".format(2*Ntot))
print("Coarse grid dimension {0}".format(Nagg*Ntest))

In [ ]:
#Computing the aggregates
for x in range(block_x):
    for t in range(block_t):
        for s in range(2):
            x0, t0 = x * x_elements, t * t_elements
            x1, t1 = (x+1) * x_elements, (t+1) * t_elements
            aggregate = x * block_t * 2 + t * 2 + s
            count = 0
            for X in range(x0,x1):
                for T in range(t0,t1):
                    i = X * Nt * 2 + T * 2 + s
                    XCoord[i] , TCoord[i], SCoord[i] = X, T, s
                    Agg[aggregate][count] = i
                    count += 1

In [ ]:
class AMG():
    def __init__(self,D,nu1,nu2):
        self.test_vectors = np.zeros((Ntest,2*Ntot),dtype=complex) 
        self.P = np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        self.PH = np.zeros((Ntest*Nagg,2*Ntot),dtype=complex) 
        self.D = csc_array(D) #Dirac matrix
        self.Dc = np.zeros((Ntest*Nagg,Ntest*Nagg),dtype=complex) 
        self.nu1 = nu1
        self.nu2 = nu2
        self.it = 0 #Parameter to measure iterations for convergence

    def get_p(self):
        return self.P, self.PH
    def get_Dc(self):
        return self.Dc
    def get_iterations(self):
        return self.it
    def get_test_vectors(self):
        return self.test_vectors

    def orthonormalize(self):
        v_chopped = [] #Columns of the interpolator
        for i in range(Ntest*Nagg):
            v = np.eye(1,Ntest*Nagg,i).reshape(Ntest*Nagg)
            v_chopped.append(self.P @ v)
        v_chopped = np.array(v_chopped)        
        for i in range(Nagg):
            for nt in range(Ntest):
                for j in range(nt):
                    #vdot(a,b) = a* . b /= b* . a = vdot(b,a)
                    proj = np.vdot(v_chopped[j * Nagg + i],v_chopped[nt * Nagg + i])
                    v_chopped[nt * Nagg + i] -= proj * v_chopped[j * Nagg + i]
                v_chopped[nt*Nagg+i] /= np.linalg.norm(v_chopped[nt*Nagg+i])
        matrix =  np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        for i in range(Ntest*Nagg):
            matrix[:,i] = v_chopped[i]
        self.P = matrix
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (self.PH @(self.D @ self.P)) #matrix multiplication

    def tv(self,Nit):  
        for i in range(Ntest): 
            for j in range(2*Ntot):
                theta = np.random.uniform(0,2*pi)
                self.test_vectors[i,j] = np.cos(theta) + 1j*np.sin(theta)
        for i in range(Ntest):
            self.test_vectors[i], exit_code = gmres(self.D, self.test_vectors[i], x0=self.test_vectors[i] ,maxiter=20)
            #self.test_vectors[i] = Kaczmarz(self.D,2*Ntot,self.test_vectors[i],self.test_vectors[i],20,1e-10)
        self.P_v() #Assemble P
        self.orthonormalize() #Orthonormalize the columns of P, it does not modify the test vectors
        print("Improving interpolator")
        for n in range(Nit):
            print("n={0} out of Nit={1}".format(n,Nit))
            for i in range(Ntest):
                self.test_vectors[i] = self.TwoGrid(1,1e-10,self.test_vectors[i],self.test_vectors[i],False)
            self.P_v() #Assemble P
            self.orthonormalize() #Orthonormalize, again it does not modify the test vectors      
             
    def P_v(self):
        #Assembling the interpolator
        for can_vec in range(Ntest*Nagg):   
            P_v = np.zeros(2*Ntot,dtype=complex)
            v = np.eye(1,Ntest*Nagg,can_vec).reshape(Ntest*Nagg)
            for j in range(Ntest*Nagg):
                k = int(j / Nagg)
                a = int(j % Nagg)
                for i in range(len(Agg[a])):
                    x, t, s = XCoord[Agg[a,i]], TCoord[Agg[a,i]], SCoord[Agg[a,i]]
                    P_v[Coords[x,t,s]]  += self.test_vectors[k,Coords[x,t,s]] * v[j]
            self.P[:,can_vec] = P_v   
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (self.PH @(self.D @ self.P)) #matrix multiplication
        
    def TwoGrid(self,max_iter,tol,x0,phi,message):
        #Solve D x = phi
        x = x0
        err = 1
        k = 0
        r0 = phi - self.D.dot(x)
        max_err = tol*np.linalg.norm(phi)
        while(k < max_iter and err > max_err):
            if self.nu1 > 0:
                x, exitCode = gmres(self.D, phi, x0=x , rtol=1e-10, atol=0.0, maxiter=nu1) 
                #x = Kaczmarz(self.D,2*Ntot,phi,x,nu1,1e-10)
            #Coarse grid correction 
            Pt_r = self.PH.dot( (phi-self.D.dot(x)))
            Dc_inv, exit_code = bicgstab(self.Dc, Pt_r,x0=Pt_r,maxiter=10000, rtol=1e-10, atol=0.0) #Dc^-1 P^H(phi-Dx)
            
            if message==True:
                rc = Pt_r - self.Dc.dot(Dc_inv)
                err_c = np.sqrt(np.real(np.vdot(rc,rc)))
                if exit_code == 0:
                    print("Bi-cgstab converged for Dc. Error {0}".format(err_c))
                else:
                    print("Bi-cgstab did NOT converge for Dc. Error {0}".format(err_c))
            x += self.P.dot(Dc_inv)
            if self.nu2>0:
                x, exitCode = gmres(self.D, phi, x0=x ,rtol=1e-10, atol=0.0, maxiter=nu2)
                #x = Kaczmarz(self.D,2*Ntot,phi,x,nu2,1e-10)
            r = phi - self.D.dot(x)
            err = np.sqrt(np.real(np.vdot(r,r))) #complex dot product
            if message == True:
                print("iteration",k+1,"error",err)
            if err<=max_err:
                self.it = k+1
                print("two-grid converged in",k+1,"iterations. Error ",err)
                return x
            k+=1
        if message == True:
            print("two-grid did not converge in",max_iter,"iterations. Error ",err)
        self.it = max_iter
        return x

    def print_p(self):
        for i in range(2*Ntot):
            for j in range(Ntest*Nagg):
                print(self.P[i,j], end = " ")
            print('')

In [ ]:
dim = Nx*Nt*2
print(dim)
D = np.zeros((dim,dim),dtype=complex)
row, col, re, im = DatosMat("build/D0.dat")
for i in range(dim*dim):
    D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
phi = np.zeros(dim,dtype=complex)
row, col, re, im = DatosMat("build/Phi0.dat")
for i in range(dim):
    phi[i] = re[i] + 1j*im[i] 
Dsparse = csc_array(D)

In [ ]:
%%timeit
x_gmres = phi #before x0
#GMRES
tol = 1e-10
print("---GMRES---")
x_gmres, exit_code = gmres(Dsparse, phi, x0=phi, rtol=1e-10, atol=0.0, restart=80,maxiter=1)
r = phi - Dsparse.dot(x_gmres)
err = np.sqrt(np.real(np.vdot(r,r)))
print("Final error {0}".format(err))

In [ ]:
Dsparse = csc_array(D)
x_gmres = phi #before x0
#GMRES
tol = 1e-10
print("---GMRES---")
for i in range(100):
    #restart is the number of iterations per restart cycle, while maxiter is the number of restarts
    #each iteration of this for loop is a full restart cycle
    x_gmres, exit_code = gmres(Dsparse, phi, x0=x_gmres, rtol=1e-10, atol=0.0, maxiter=1)
    r = phi - Dsparse.dot(x_gmres)
    err = np.sqrt(np.real(np.vdot(r,r)))
    print(i+1,err,exit_code)
    if err<=tol*np.linalg.norm(phi):
        print("GMRES converged in {0} iterations".format(i+1))
        break
#bi-cgstab
print("---Bi-cgstab---")
x_bi = phi #before x0
x_bi, exit_code = bicgstab(Dsparse, phi,x0=x_bi,maxiter=500, rtol=1e-10)
if exit_code == 0:
    print("Converged")
else:
    print("Did not converge")
r = phi - Dsparse.dot(x_bi)
err = np.sqrt(np.real(np.vdot(r,r)))
print("Bi-cgstab error {0}".format(err))

In [ ]:
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)
nu1, nu2 = 0, 2
Nit = 1
amg = AMG(D,nu1,nu2)
amg.tv(Nit)
P, PH = amg.get_p()
max_iter = 100
tol = 1e-10
x_sol = amg.TwoGrid(max_iter,tol,x0,phi,True)

In [ ]:
for i in range(2*Ntot):
    line_new = '{:>12} {:>12} {:>12}'.format(x_sol[i], x_bi[i], x_gmres[i])
    print(line_new)

### Some tests for a two-grid method using GRMES as smoother

In [ ]:
dim = Nx*Nt*2
D = np.zeros((dim,dim),dtype=complex)
phi = np.zeros(dim,dtype=complex)
x0 = np.zeros(2*Ntot,dtype=complex)
iterations = []
for conf in range(50):
    row, col, re, im = DatosMat("build/D{0}.dat".format(conf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    row, col, re, im = DatosMat("build/Phi{0}.dat".format(conf))
    for i in range(dim):
        phi[i] = re[i] + 1j*im[i] 
    for i in range(2*Ntot):  
        theta = np.random.uniform(0,2*pi)
        x0[i] = np.cos(theta) + 1j*np.sin(theta)
    nu1, nu2 = 0, 2
    Nit = 5
    print("Conf #{0}".format(conf))
    amg = AMG(D,nu1,nu2)
    amg.tv(Nit)
    P, PH = amg.get_p()
    max_iter = 100
    tol = 1e-10
    x_sol = amg.TwoGrid(max_iter,tol,x0,phi,False)
    iterations.append(amg.get_iterations())
iterations = np.array(iterations)
print("Number of iterations: {0} +- {1}".format(np.round(np.mean(iterations),4), np.round(np.std(iterations)/np.sqrt(50),4  ) ))

### Tests using GMRES

In [ ]:
dim = Nx*Nt*2
D = np.zeros((dim,dim),dtype=complex)
phi = np.zeros(dim,dtype=complex)
x0 = np.zeros(2*Ntot,dtype=complex)
tol = 1e-10
iterations = []
for conf in range(8):
    row, col, re, im = DatosMat("build/D{0}.dat".format(conf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    row, col, re, im = DatosMat("build/Phi{0}.dat".format(conf))
    for i in range(dim):
        phi[i] = re[i] + 1j*im[i] 
    for i in range(2*Ntot):  
        theta = np.random.uniform(0,2*pi)
        x0[i] = np.cos(theta) + 1j*np.sin(theta)
    Dsparse = csc_array(D)
    x_gmres = x0
    #GMRES
    print("Conf #{0}".format(conf))
    max_err = tol*np.linalg.norm(phi)
    err = 1
    k = 0
    while k<1000 and err>max_err:
        x_gmres, exit_code = gmres(Dsparse, phi, x0=x_gmres, rtol=1e-10, atol=0.0, maxiter=1)
        r = phi - Dsparse.dot(x_gmres)
        err = np.sqrt(np.real(np.vdot(r,r)))
        if err<=max_err:
            print("GMRES converged in {0} iterations".format(k+1))
            iterations.append([k+1])
        k += 1
iterations = np.array(iterations)
print("Number of iterations: {0} +- {1}".format(np.round(np.mean(iterations),4), np.round(np.std(iterations)/np.sqrt(50),4  ) ))       

### Testing Kaczmarz as a solver

In [ ]:
dim = Nx*Nt*2
print(dim)
D = np.zeros((dim,dim),dtype=complex)
row, col, re, im = DatosMat("build/D0.dat")
for i in range(dim*dim):
    D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
phi = np.zeros(dim,dtype=complex)
row, col, re, im = DatosMat("build/Phi0.dat")
for i in range(dim):
    phi[i] = re[i] + 1j*im[i] 

#Initial solution
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)

Dsparse = csc_array(D)
tol = 1e-10
max_iter = 500
x = deepcopy(x0)
for i in range(max_iter):
    x = Kaczmarz(D,2*Ntot,phi,x,1,1e-10)
    r = phi - Dsparse.dot(x)
    err = np.sqrt(np.real( np.vdot(r,r) ))
    print(err)
    if err<=tol:
        print("Kaczmarz converged in {0} iterations".format(i+1))
        break
print(x[0])
#x = deepcopy(x0)
#x=Kaczmarz(D,2*Ntot,phi,x,5,1e-10)
#print(x[0])

## Memory occupied by the Dirac matrix in the Schwinger model

In [ ]:
Ns = 100 #Square lattice dimensions
DIM = 2*Ns**2
byts = 2*8*(DIM)**2
print("bytes = {0}\n Mb = {1}\n Gb = {2}".format(byts,byts/1e6,byts/1e9))